In [ ]:
import time
import logging
import csv
import os
from datetime import datetime
from logging.handlers import RotatingFileHandler

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openlocationcode import openlocationcode as olc

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        RotatingFileHandler('scraper_review.log', maxBytes=5*1024*1024, backupCount=2, encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
def setup_driver(headless=False):
    """Thiết lập và cấu hình trình duyệt Chrome.

    Args:
        headless (bool): Chạy trình duyệt ở chế độ không giao diện nếu True.

    Returns:
        WebDriver: Trình duyệt Chrome đã được cấu hình.

    Raises:
        Exception: Nếu thiết lập trình duyệt thất bại.
    """
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    if headless:
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", {
            "latitude": 16.0544,
            "longitude": 108.2022,
            "accuracy": 100
        })
        logger.info("Thiết lập trình duyệt thành công.")
        return driver
    except Exception as e:
        logger.error(f"Lỗi khi thiết lập trình duyệt: {e}")
        raise

In [ ]:
def click(driver, element, wait_time=2, retries=3):
    for attempt in range(retries):
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable(element))
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            driver.execute_script("arguments[0].click();", element)
            time.sleep(wait_time)
            logger.info("Click thành công.")
            return True
        except (ElementClickInterceptedException, TimeoutException) as e:
            logger.warning(f"Click thất bại (lần {attempt + 1}/{retries}): {e}")
            time.sleep(1)
    logger.error(f"Click thất bại sau {retries} lần thử.")
    return False

In [ ]:
# Hàm cuộn và click nút "Xem thêm"
def scroll_and_click_more(driver, scrollable_div, max_scrolls=20):
    last_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
    scroll_count = 0
    reviews_loaded = 0
    
    while scroll_count < max_scrolls:
        current_reviews = len(driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium"))
        logger.info(f"Đã tải {current_reviews} đánh giá.")

        more_buttons = driver.find_elements(By.CSS_SELECTOR, "button.w8nwRe.kyuRq")
        for btn in more_buttons:
            if click(driver, btn, wait_time=0.5):
                time.sleep(0.5)
        
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
        time.sleep(1.5)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
        new_reviews = len(driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium"))
        
        if new_height == last_height and new_reviews == reviews_loaded:
            scroll_count += 1
            if scroll_count >= 3:
                logger.info("Không còn đánh giá mới, dừng cuộn.")
                break
        else:
            scroll_count = 0
            logger.info(f"Tải thêm dữ liệu mới. Chiều cao mới: {new_height}, Đánh giá mới: {new_reviews}")
        
        last_height = new_height
        reviews_loaded = new_reviews

In [ ]:
# Định nghĩa fieldnames toàn cục (đã có trong save_links, nhưng đảm bảo đồng bộ)
FIELDNAMES = [
    "Restaurant_id", "Url", "Restaurant_name", "Restaurant_type", "Rating_average",
    "Num_of_reviews", "Phone", "Price_level", "Address",
    "Latitude", "Longitude", "Crawl_date"
]

def init_csv(output_file="restaurants.csv"):
    """Tạo file CSV nếu chưa tồn tại."""
    if not os.path.isfile(output_file):
        try:
            with open(output_file, mode="w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
                writer.writeheader()
            logger.info(f"Tạo file CSV mới: {output_file}")
        except Exception as e:
            logger.error(f"Lỗi khi tạo file CSV {output_file}: {e}")
            raise
    else:
        logger.info(f"File CSV {output_file} đã tồn tại.")

In [ ]:
# Hàm thu thập đánh giá
def scrape_review(driver, restaurant_url, restaurant_id):
    reviews_data = []
    try:
        logger.info(f"Đang xử lý đánh giá cho URL: {restaurant_url}")
        driver.get(restaurant_url)
        
        # Tìm tab đánh giá
        try:
            reviews_tab = driver.find_elements(By.XPATH, "//div[text()='Reviews']")
            if not reviews_tab:
                logger.warning("Không tìm thấy tab đánh giá.")
                return reviews_data
            click(driver, reviews_tab[0])
            scrollable_divs = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf"))
            )
            if scrollable_divs:
                scroll_and_click_more(driver, scrollable_divs[0])
            
            review_containers = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium")
            logger.info(f"Tìm thấy {len(review_containers)} đánh giá.")
            
            for container in review_containers:
                try:
                    # Lấy thông tin đánh giá
                    reviewer_name = container.find_element(By.CSS_SELECTOR, "div.d4r55").text or None
                    reviewer_info = container.find_element(By.CSS_SELECTOR, "div.RfnDt").text or None
                    
                    # Lấy điểm đánh giá
                    rating_value = None
                    try:
                        rating_elem = container.find_element(By.CSS_SELECTOR, "span[aria-label]")
                        rating_text = rating_elem.get_attribute('aria-label').split()[0]
                        rating_value = float(rating_text.replace(',', '.'))
                    except:
                        pass
                    
                    # Lấy thời gian và nội dung đánh giá
                    review_time = container.find_element(By.CSS_SELECTOR, "span.rsqaWe").text or None
                    review_text = container.find_element(By.CSS_SELECTOR, "span.wiI7pd").text or None
                    
                    # Lấy ngôn ngữ
                    language = None
                    try:
                        language_elem = container.find_element(By.CSS_SELECTOR, "div.oqftme")
                        text = language_elem.text
                        if "(" in text and ")" in text:
                            language = text.split("(")[-1].replace(")", "").strip()
                    except:
                        pass
                    
                    # Lấy các tiêu chí đánh giá
                    rating_elements = container.find_elements(By.CSS_SELECTOR, "span.RfDO5c")
                    service_rating = food_rating = atmosphere_rating = service_type = meal_type = None
                    
                    for elem in rating_elements:
                        text = elem.text.strip()
                        if "Service:" in text:
                            service_rating = text.split(":")[-1].strip()
                        elif "Food:" in text:
                            food_rating = text.split(":")[-1].strip()
                        elif "Atmosphere:" in text:
                            atmosphere_rating = text.split(":")[-1].strip()
                        elif text in ["Dine in", "Takeout"]:
                            service_type = text
                        elif text in ["Breakfast", "Lunch", "Dessert", "Brunch", "Dinner", "Seating"]:
                            meal_type = text
                    
                    # Xử lý trường hợp review_text rỗng
                    if not review_text or review_text.strip() == "":
                        if rating_value is not None:
                            review_text = f"Rated {rating_value}"
                            logger.info(f"Đã gán review_text thành 'Rated {rating_value}' vì không có nội dung đánh giá.")
                        else:
                            logger.info("Không có nội dung đánh giá và rating_value, giữ review_text là None.")
                    
                    # Chuẩn hóa dữ liệu
                    reviewer_name = reviewer_name if reviewer_name and len(reviewer_name) <= 500 else reviewer_name[:500] if reviewer_name else None
                    reviewer_info = reviewer_info if reviewer_info and len(reviewer_info) <= 500 else reviewer_info[:500] if reviewer_info else None
                    review_time = review_time if review_time and len(review_time) <= 100 else review_time[:100] if review_time else None
                    language = language if language and len(language) <= 100 else language[:100] if language else None
                    
                    # Thêm vào danh sách đánh giá
                    reviews_data.append({
                        "restaurant_id": restaurant_id,
                        "reviewer_name": reviewer_name,
                        "reviewer_info": reviewer_info,
                        "rating": rating_value,
                        "review_time": review_time,
                        "review_text": review_text,
                        "service_rating": service_rating,
                        "food_rating": food_rating,
                        "atmosphere_rating": atmosphere_rating,
                        "service_type": service_type,
                        "meal_type": meal_type,
                        "language": language,
                        "created_at": pd.Timestamp.now()
                    })
                except Exception as e:
                    logger.warning(f"Lỗi khi xử lý đánh giá: {e}")
                    continue
            
            logger.info("Đã thu thập đánh giá thành công.")
        except Exception as e:
            logger.error(f"Lỗi khi xử lý đánh giá: {e}")
        
        return reviews_data
    except Exception as e:
        logger.error(f"Lỗi khi truy cập URL {restaurant_url}: {e}")
        return reviews_data

In [ ]:
def update_details(driver, output_file="restaurants.csv", batch_size=10):
    """Đọc file CSV, scrape lại từng link và cập nhật nếu có thay đổi.
       Ghi file một lần sau khi xử lý toàn bộ danh sách để tránh xung đột.
       Trả về số lượng nhà hàng được cập nhật.

    Args:
        driver: WebDriver instance.
        output_file: Đường dẫn file CSV (mặc định: restaurants.csv).
        batch_size: Số lượng nhà hàng mỗi lần log tiến độ (mặc định: 10).

    Returns:
        int: Số lượng nhà hàng được cập nhật.
    """
    wait = WebDriverWait(driver, 10)
    
    # Đọc dữ liệu hiện có
    rows = []
    try:
        with open(output_file, mode="r", encoding="utf-8-sig") as f:
            reader = csv.DictReader(f)
            rows = list(reader)
    except Exception as e:
        logger.error(f"Lỗi khi đọc file CSV {output_file}: {e}")
        return 0

    updated = 0
    total = len(rows)
    updated_rows = rows.copy()  # Sao chép để cập nhật dữ liệu

    for i, row in enumerate(updated_rows):
        url = row.get("Url", "")
        if not url:
            logger.warning(f"Bỏ qua dòng {i+1}: Không có URL")
            continue
        logger.info(f"Scraping {i+1}/{total}: {url}")
        try:
            driver.get(url)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf")))
            
            data = scrape_review(driver)
            if data:
                has_change = False
                for k in data.keys():
                    if k == "Crawl_date":
                        continue
                    new_val = str(data.get(k, ""))
                    old_val = str(row.get(k, ""))
                    if new_val.strip() == "" and old_val.strip() != "":
                        # Nếu mới rỗng nhưng cũ có giá trị, giữ nguyên cũ
                        data[k] = old_val
                    elif new_val != old_val:
                        has_change = True
                
                if has_change:
                    row.update({k: data.get(k, row.get(k, "")) for k in data.keys() if k != "Crawl_date"})
                    row["Crawl_date"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    updated += 1
                    logger.info(f"Cập nhật thay đổi cho nhà hàng: {data.get('Restaurant_name', url)}")
                else:
                    logger.info(f"Không có thay đổi cho nhà hàng: {data.get('Restaurant_name', url)}")
        except (TimeoutException, NoSuchElementException) as e:
            logger.error(f"Lỗi scrape {url}: {e}")
            continue

        if (i + 1) % batch_size == 0:
            logger.info(f"Đã xử lý {i+1}/{total} nhà hàng...")

    # Ghi file CSV một lần sau khi xử lý tất cả
    try:
        with open(output_file, mode="a", encoding="utf-8-sig") as f:
            pass  # Kiểm tra quyền ghi
        with open(output_file, mode="w", newline="", encoding="utf-8-sig") as f:
            writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
            writer.writeheader()
            writer.writerows(updated_rows)
        logger.info(f"Hoàn thành! Đã cập nhật {updated} nhà hàng có thay đổi trong {output_file}")
    except PermissionError as e:
        logger.error(f"Lỗi khi ghi file CSV {output_file}: {e}. Vui lòng kiểm tra xem file có đang được mở bởi chương trình khác không.")
        return updated
    except Exception as e:
        logger.error(f"Lỗi không xác định khi ghi file CSV {output_file}: {e}")
        return updated

    return updated

In [ ]:
def main(search_url="https://www.google.com/maps/search/Restaurants+in+Da+Nang", output_file="restaurants.csv", batch_size=10, headless=False):
    """Chạy chương trình crawl Google Maps.

    Args:
        search_url: URL tìm kiếm Google Maps (mặc định: nhà hàng ở Đà Nẵng).
        output_file: Đường dẫn file CSV (mặc định: restaurants.csv).
        batch_size: Số lượng nhà hàng mỗi lần log tiến độ (mặc định: 10).
        headless: Chạy trình duyệt ở chế độ không giao diện nếu True (mặc định: False).
    """
    start_time = datetime.now()
    logger.info(f"Bắt đầu crawl: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    driver = None
    try:
        logger.info("Khởi tạo trình duyệt...")
        driver = setup_driver(headless=headless)

        logger.info(f"Mở URL: {search_url}")
        driver.get(search_url)

        # B1: Tạo file CSV
        logger.info(f"Khởi tạo file CSV: {output_file}")
        init_csv(output_file)

        # B2: Crawl danh sách link
        try:
            wait = WebDriverWait(driver, 10)
            scrollable_div = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='feed']"))
            )
            logger.info("Bắt đầu cuộn để tải thêm nhà hàng...")
            scroll_until_end(driver, scrollable_div)
            logger.info("Lưu danh sách link vào CSV...")
            save_links(driver, output_file)
        except TimeoutException as e:
            logger.error(f"Không thể tìm thấy danh sách nhà hàng: {e}")
            raise

        # B3: Cập nhật chi tiết nhà hàng
        logger.info("Bắt đầu cập nhật chi tiết nhà hàng...")
        updated = update_details(driver, output_file, batch_size)
        logger.info(f"Hoàn thành cập nhật dữ liệu! Đã cập nhật {updated} nhà hàng.")

    except (TimeoutException, WebDriverException) as e:
        logger.error(f"Lỗi trong quá trình thực thi: {e}")
        raise
    finally:
        if driver:
            driver.quit()
            logger.info("Đã đóng trình duyệt.")
        end_time = datetime.now()
        logger.info(f"Kết thúc crawl: {end_time.strftime('%Y-%m-%d %H:%M:%S')}, thời gian chạy: {end_time - start_time}, tổng số nhà hàng cập nhật: {updated}")

if __name__ == "__main__":
    main()